<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/Lab_11_SQL_Connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Tutorial

In this tutorial, we will refresh some knowledge of SQL, and learn how to interact directly between Python and SQL.

For this, we will use [SQLite](https://www.sqlite.org), a library that allows connecting to small databases. SQLite is a great utility: It allows creating efficient databases on-the-fly, and it is oriented towards apps, applications, and in-place data analytics. A great summary of case uses for SQLite can be found [here](https://www.sqlite.org/whentouse.html).

(Note: SQLite should be installed in your PC. If not, download the binaries from [here](https://www.sqlite.org/download.html) and save them in your notebook's folder).

The provided file comes with the [Chinook sample database](http://www.sqlitetutorial.net/sqlite-sample-database/). This  database simulates a music business, and its schema is represented by:

![Schema](http://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

We will use this database to refresh our knowledge of SQL, and then to create Pandas datasets directly from any database. This is a common operation when doing data preprocessing.


SQL is a very extensive language, but its mastery comes from practice. We will only see some of the tutorial activities today, so I advice you to go to [the SQLite tutorial](http://www.sqlitetutorial.net/) and go through some of the most advanced characteristics, and also go through the full [SQL reference on W3School](https://www.w3schools.com/sql/sql_quickref.asp).

## Reading Data

First we import the data we will use, from the [SQLite tutorial](http://www.sqlitetutorial.net/sqlite-sample-database/).

In [2]:
!wget http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
!unzip chinook.zip

--2019-05-09 13:17:23--  http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 66.147.244.187
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|66.147.244.187|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  1.72MB/s    in 0.2s    

2019-05-09 13:17:23 (1.72 MB/s) - ‘chinook.zip’ saved [305596/305596]

Archive:  chinook.zip
  inflating: chinook.db              


Now we import the sqlite package, called sqlite3

In [0]:
import sqlite3

The basic object in a SQLite query is the *Connection*. A Connection object links Python with the database.

In [0]:
conn = sqlite3.connect('chinook.db')

Now we are ready to create our first query. Let's retrieve the ID, name, composer and price for all tracks in our dataset. We first will create the SQL query into a variable, and then simply execute the query into a *Cursor*, which is simply a pointer to the output of the query.

The basic structure of a SQL SELECT query is:

```
SELECT [fields]
FROM [table]
WHERE [condition]
ORDER BY [field]
```

In [0]:
query = "SELECT trackid, name, composer, unitprice FROM tracks;"
out = conn.execute(query)

We can now show the results of the query with the method *fetchall()*. Note that, as *out* is a cursor, once we do this we will have reached the end of the query, thus we will need to run it again to get new elements. We can also use the *fetchmany* or *fetchone* methods to get segments of the query, or even iterate over the results if necessary.

In [5]:
out.fetchall()

[(1,
  'For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson',
  0.99),
 (2, 'Balls to the Wall', None, 0.99),
 (3,
  'Fast As a Shark',
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman',
  0.99),
 (4,
  'Restless and Wild',
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman',
  0.99),
 (5, 'Princess of the Dawn', 'Deaffy & R.A. Smith-Diesel', 0.99),
 (6,
  'Put The Finger On You',
  'Angus Young, Malcolm Young, Brian Johnson',
  0.99),
 (7, "Let's Get It Up", 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (8, 'Inject The Venom', 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (9, 'Snowballed', 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (10, 'Evil Walks', 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (11, 'C.O.D.', 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (12, 'Breaking The Rules', 'Angus Young, Malcolm Young, Brian Johnson', 0.99),
 (13,
  'Night Of The Long Knives',
  'Angus Young, 

In [0]:
# We reached EOF, so this is empty.
out.fetchone()

Let's create a slightly more complex query. For example, let's create a query that returns all the songs written by AC/DC. We will at this point use a more secure way to write a query, by passing the artist name as a wildcard character.

In [0]:
artist = ('AC/DC',)
query = "SELECT trackid, name, composer, unitprice FROM tracks WHERE composer=?;"
out = conn.execute(query, artist)

In [8]:
out.fetchall()

[(15, 'Go Down', 'AC/DC', 0.99),
 (16, 'Dog Eat Dog', 'AC/DC', 0.99),
 (17, 'Let There Be Rock', 'AC/DC', 0.99),
 (18, 'Bad Boy Boogie', 'AC/DC', 0.99),
 (19, 'Problem Child', 'AC/DC', 0.99),
 (20, 'Overdose', 'AC/DC', 0.99),
 (21, "Hell Ain't A Bad Place To Be", 'AC/DC', 0.99),
 (22, 'Whole Lotta Rosie', 'AC/DC', 0.99)]

With this notation, we are reducing the risk of [SQL Injection attacks](https://en.wikipedia.org/wiki/SQL_injection), which is by far the most common hack out there. We reduce the risk because we can run checks on the variable in order to clean it from potential attacks. This is called [input sanitization](https://download.oracle.com/oll/tutorials/SQLInjection/html/lesson1/les01_tm_ovw3.htm).

## Joining tables

The most common operation in database querying is to join tables. This way, we can create views (queries) that put together the information that we need efficiently. Most join operations that we will use are called "inner joins", that is, they take the information from the tables and return the ones that match in either both or just one of the tables. There are also "outer joins" that returns all information in two tables, and using negations we can get the ones that do not match.

![Join types](http://1.bp.blogspot.com/-_PHkf1f9Vpk/UHGgfNrLxEI/AAAAAAAAAUk/NTTqGCCefjw/s1600/sqljoins2.png)

Let's create a query that gives us the album title as well as the track title and composer for all authors.

In [9]:
query = "SELECT T.trackid, T.name, T.composer, A.title, T.unitprice FROM tracks AS T INNER JOIN Albums as A "
query += "ON T.AlbumID = A.AlbumID"
out = conn.execute(query)
out.fetchall()

[(1,
  'For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (6,
  'Put The Finger On You',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (7,
  "Let's Get It Up",
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (8,
  'Inject The Venom',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (9,
  'Snowballed',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (10,
  'Evil Walks',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (11,
  'C.O.D.',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (12,
  'Breaking The Rules',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salu

As we can see, the songs that do not have any album do not appear (i.e. songs 2 - 5). A *LEFT JOIN* will give us all songs from Tracks, and complete the information from albums if available.

In [10]:
query = "SELECT T.trackid, T.name, T.composer, A.title, T.unitprice FROM tracks as T LEFT JOIN Albums as A "
query += "ON T.AlbumID = A.AlbumID"
out = conn.execute(query)
out.fetchall()

[(1,
  'For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (2, 'Balls to the Wall', None, 'Balls to the Wall', 0.99),
 (3,
  'Fast As a Shark',
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman',
  'Restless and Wild',
  0.99),
 (4,
  'Restless and Wild',
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman',
  'Restless and Wild',
  0.99),
 (5,
  'Princess of the Dawn',
  'Deaffy & R.A. Smith-Diesel',
  'Restless and Wild',
  0.99),
 (6,
  'Put The Finger On You',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (7,
  "Let's Get It Up",
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (8,
  'Inject The Venom',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (9,
  'Snowballed',
  'Angus Young, Malcolm Young, Brian Johns

What would be the result of a Right Join?

## Grouping Operations

Most of the time we want to calculate summary operations over the results of a query, which usually requires nesting subsequent joins. We can apply several summary operations, such as SUM, AVG, COUNT, etc.

For this, we need the GROUP BY expression. It tells SQL to aggregate results, but only those grouped by one or more values. For example, let's count the number of songs per album, for those songs with an album present.

In [11]:
query = "SELECT A.title, COUNT(T.unitprice) AS NrSongs FROM tracks as T INNER JOIN Albums as A "
query += "ON T.AlbumID = A.AlbumID"
query += " GROUP BY A.title"
out = conn.execute(query)
out.fetchall()

[('...And Justice For All', 9),
 ('20th Century Masters - The Millennium Collection: The Best of Scorpions',
  12),
 ('A Copland Celebration, Vol. I', 1),
 ('A Matter of Life and Death', 11),
 ('A Real Dead One', 12),
 ('A Real Live One', 11),
 ('A Soprano Inspired', 1),
 ('A TempestadeTempestade Ou O Livro Dos Dias', 15),
 ('A-Sides', 17),
 ('Ace Of Spades', 15),
 ('Achtung Baby', 12),
 ('Acústico', 22),
 ('Acústico MTV', 21),
 ('Acústico MTV [Live]', 17),
 ('Adams, John: The Chairman Dances', 1),
 ('Adorate Deum: Gregorian Chant from the Proper of the Mass', 1),
 ('Afrociberdelia', 23),
 ('Album Of The Year', 12),
 ('Alcohol Fueled Brewtality Live! [Disc 1]', 13),
 ('Alcohol Fueled Brewtality Live! [Disc 2]', 5),
 ("All That You Can't Leave Behind", 11),
 ('Allegri: Miserere', 1),
 ('American Idiot', 13),
 ('Angel Dust', 14),
 ('Ao Vivo [IMPORT]', 19),
 ('Appetite for Destruction', 12),
 ('Aquaman', 1),
 ('Are You Experienced?', 17),
 ('Armada: Music from the Courts of England and Sp

Let's do a more complex calculation. For example, let's get the total price per album. This requires first constructing a table with album, song name, and prices, such as the one we did before, and then grouping the results by album, adding the total price per song.

For this, temporal tables can make our life easier, and more efficient. A temporal table is an intermediate table that will store results for a short period of time (usually a sequence of operations). The following code creates a temporary table called *AlbumSongs* that holds the output of the query we did before. We will use the *CREATE TABLE* expression, adding the *TEMPORARY* modifier that tells SQLite that the table should dissappear later.


In [0]:
query = "CREATE TEMPORARY TABLE AlbumSongs AS "
query += "SELECT T.trackid, T.name, T.composer, A.title, T.unitprice FROM tracks as T LEFT JOIN Albums as A "
query += "ON T.AlbumID = A.AlbumID"
out = conn.execute(query)

Now there is a table called AlbumSongs with the output of our query. This table will dissapear once we close our connection to the SQLite database (*conn.close()*).

In [13]:
query = "SELECT * FROM AlbumSongs"
out = conn.execute(query)
out.fetchall()

[(1,
  'For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (2, 'Balls to the Wall', None, 'Balls to the Wall', 0.99),
 (3,
  'Fast As a Shark',
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman',
  'Restless and Wild',
  0.99),
 (4,
  'Restless and Wild',
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman',
  'Restless and Wild',
  0.99),
 (5,
  'Princess of the Dawn',
  'Deaffy & R.A. Smith-Diesel',
  'Restless and Wild',
  0.99),
 (6,
  'Put The Finger On You',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (7,
  "Let's Get It Up",
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (8,
  'Inject The Venom',
  'Angus Young, Malcolm Young, Brian Johnson',
  'For Those About To Rock We Salute You',
  0.99),
 (9,
  'Snowballed',
  'Angus Young, Malcolm Young, Brian Johns

Now we can simply create a summary query that returns the album, and we will add the number of songs and total price. We will save this table as a permanent table *AlbumStats*.

In [15]:
# AS Name saves the output as a table with name Name
query = "SELECT Title, COUNT(*) AS NrSongs, SUM(unitprice) AS TotalPrice FROM AlbumSongs "
query += "GROUP BY Title"
out = conn.execute(query)
out.fetchall()

[('...And Justice For All', 9, 8.91),
 ('20th Century Masters - The Millennium Collection: The Best of Scorpions',
  12,
  11.88),
 ('A Copland Celebration, Vol. I', 1, 0.99),
 ('A Matter of Life and Death', 11, 10.89),
 ('A Real Dead One', 12, 11.88),
 ('A Real Live One', 11, 10.89),
 ('A Soprano Inspired', 1, 0.99),
 ('A TempestadeTempestade Ou O Livro Dos Dias', 15, 14.850000000000001),
 ('A-Sides', 17, 16.830000000000002),
 ('Ace Of Spades', 15, 14.850000000000001),
 ('Achtung Baby', 12, 11.88),
 ('Acústico', 22, 21.779999999999994),
 ('Acústico MTV', 21, 20.789999999999996),
 ('Acústico MTV [Live]', 17, 16.830000000000002),
 ('Adams, John: The Chairman Dances', 1, 0.99),
 ('Adorate Deum: Gregorian Chant from the Proper of the Mass', 1, 0.99),
 ('Afrociberdelia', 23, 22.769999999999992),
 ('Album Of The Year', 12, 11.88),
 ('Alcohol Fueled Brewtality Live! [Disc 1]', 13, 12.870000000000001),
 ('Alcohol Fueled Brewtality Live! [Disc 2]', 5, 4.95),
 ("All That You Can't Leave Behind"

## Modifying Data

Many times we want to create tables and add some data as we go along. The instructions INSERT INTO allow for easy insertion of new cases into our dataset. Let's start by creating a new table out of our results.

In [0]:
query = "CREATE TEMPORARY TABLE AlbumStats AS SELECT Title, COUNT(*) AS NrSongs, SUM(unitprice) AS TotalPrice FROM AlbumSongs "
query += "GROUP BY Title"
out = conn.execute(query)

And now, let's add the album "MANG3073", that comes with 20 songs and costs £11.99 into our temporary table.

In [0]:
query = "INSERT INTO AlbumStats (Title, NrSongs, TotalPrice) VALUES ('MANG3073', 20, 11.99)"
out = conn.execute(query)

In [19]:
query = "SELECT * FROM AlbumStats WHERE Title = 'MANG3073'"
out = conn.execute(query)
out.fetchall()

[('MANG3073', 20, 11.99), ('MANG3073', 20, 11.99)]

Our album is now available in the database!

Finally, we can delete elements from tables using a similar syntax as SELECT.

In [0]:
query = "DELETE FROM AlbumStats WHERE Title = 'MANG3073'"
out = conn.execute(query)

In [21]:
query = "SELECT * FROM AlbumStats WHERE Title = 'MANG3073'"
out = conn.execute(query)
out.fetchall()

[]

And we can even delete whole tables, or columns, with the instruction "DROP". For example, we do not need the AlbumSongs table any longer, we can delete it (and free memory) using the following code.

In [0]:
query = "DROP TABLE AlbumSongs"
out = conn.execute(query)

Once you are done working with a database, it is always a good idea to close it.

In [0]:
conn.close()

## Pandas & Database Connections

The above is a common way to create databases on-the-fly for applications of any type, or for local preprocessing of data (i.e. loading a CSV and saving it to a more efficient database). The second common usecase is directly preprocess data in the original database, and then import the output as a Pandas dataset.

For this, we will use the far more advanced [SQLAlchemy library](http://www.sqlalchemy.org/library.html#tutorials), which will allow for complex operations in databases directly from Python.

This approach is far more common when working over a static, enterprise level, database. A typical pipeline could include:

1. Create a SQL Alchemy connection to a main server.
2. Create a SQLite database locally, using either SQLite or directly SQLAlchemy.
3. Generate operations over the master database, such that they are outputted into the local database.
4. Import results into a Pandas dataframe.

SQLAlchemy is a very sophisticated software. We will simply use the connection property of it, but you are invited to learn more about it. For a detailed explanation of why data scientists should know SQLAlchemy, read [this blog post](http://danielweitzenfeld.github.io/passtheroc/blog/2014/10/12/datasci-sqlalchemy/).

SQLAlchemy comes preinstalled in Colab, and in many images of data-oriented Python services.

Now we create a connection to our SQLite Chinook dataset. We start by creating an [Engine](https://docs.sqlalchemy.org/en/13/core/engines.html), which is simply telling SQLAlchemy which database engine and database are we using, and then create a connection to this database.

In [0]:
# Import the function
from sqlalchemy import create_engine

# Create the engine. Will look in the folder the notebook is in.
ChinEngine =create_engine('sqlite:///chinook.db')

# Create the connection which will execute the queries.
conn2 = ChinEngine.connect()

And now we can use Pandas to directly run a query and return the output as a dataset. Let's return for example the AlbumSongs dataset to Pandas.

In [0]:
query = "SELECT T.trackid, T.name, T.composer, A.title, T.unitprice FROM tracks as T LEFT JOIN Albums as A "
query += "ON T.AlbumID = A.AlbumID"

In [27]:
import pandas as pd
pd.read_sql(query, conn2)

,TrackId,Name,Composer,Title,UnitPrice
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99
1,2,Balls to the Wall,None,Balls to the Wall,0.99
2,3,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Restless and Wild,0.99
3,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Restless and Wild,0.99
4,5,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,Restless and Wild,0.99
5,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99
6,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99
7,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99
8,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99
9,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",For Those About To Rock We Salute You,0.99


SQL Alchemy supports directly executing the queries using the *execute* method over the connection, just like SQLite, but its strengths are in the multiple connections it support, and the [Object Relational Mapper](http://docs.sqlalchemy.org/en/latest/orm/), or a way to programatically represent tables as objects. This escapes the objectives of this module, but it can be worth it to learn if you are working with tables directly from Python. We will see more uses of this package next week.